In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Reading datasets
df = pd.read_csv('Dataset.csv', names=['user_id', 'item_id', 'rating', 'timestamp'], header=0)
movie_titles_df = pd.read_csv('Movie_Id_Titles.csv', names=['item_id', 'title'], header=0)

In [ ]:
# Create a dictionary to map item_id to movie title
movie_titles_dict = dict(zip(movie_titles_df.item_id, movie_titles_df.title))

# Group data by user and store as a dictionary where each user has a dictionary of movie ratings
groupedData = df.groupby('user_id').apply(lambda x: dict(zip(x.item_id, x.rating))).to_dict()

In [ ]:
# Functions for various menu options
def countUsers(groupedData):
    """Print the total number of users."""
    print(f"Total users: {len(groupedData)}")

def countMovies(groupedData):
    """Count the total number of distinct movies rated by all users."""
    moviesSet = set()
    for movies in groupedData.values():
        moviesSet.update(movies.keys())
    print(f"Total movies: {len(moviesSet)}")
    return len(moviesSet)

def displayUser(groupedData, userID):
    """Display movie ratings for a particular user."""
    user_movies = groupedData.get(userID, "User not found!")
    if user_movies == "User not found!":
        print(user_movies)
    else:
        print(f"Movies rated by User {userID}:")
        for movie_id, rating in user_movies.items():
            print(f"Movie: {movie_titles_dict.get(movie_id, 'Unknown Movie')}, Rating: {rating}")
    return user_movies

def displayMovie(movie_titles_dict, movieID):
    """Display the title of a movie given its movie ID."""
    movie_title = movie_titles_dict.get(movieID, "Movie not found!")
    print(movie_title)
    return movie_title

def recommendMovies(groupedData, userID, n, movie_titles_dict):
    """Recommend top N movies for a given user using collaborative filtering."""
    if userID not in groupedData:
        print("User not found!")
        return []

    # Create a DataFrame with users as rows and movies as columns
    ratings_df = pd.DataFrame(groupedData).fillna(0)

    # Compute similarity between users
    user_similarity = cosine_similarity(ratings_df)
    user_similarity_df = pd.DataFrame(user_similarity, index=ratings_df.index, columns=ratings_df.index)

    # Get similar users, excluding the user themselves
    similar_users = user_similarity_df[userID].sort_values(ascending=False)[1:]

    # Recommend movies that similar users liked, but the current user hasn't watched
    user_movies = set(groupedData[userID].keys())
    recommendations = {}

    for sim_user in similar_users.index:
        if sim_user in groupedData :
            for movie, rating in groupedData[sim_user].items():
                if movie not in user_movies:
                    recommendations[movie] = recommendations.get(movie, 0) + rating * similar_users[sim_user]

    # Recommend top N movies
    top_recommendations = sorted(recommendations.items(), key=lambda x: -x[1])[:n]

    # Return the movie titles instead of item_id
    recommended_movie_titles = [movie_titles_dict.get(movie, "Unknown Movie") for movie, score in top_recommendations]

    # Display the recommendations
    print(f"Recommended Movies for User {userID}:")
    for movie in recommended_movie_titles:
        print(movie)

    return recommended_movie_titles

In [ ]:
while True:
    print("\nMenu:")
    print("1. Count Users")
    print("2. Count Movies")
    print("3. Display User Information")
    print("4. Display Movie Name")
    print("5. Recommend Movies")
    print("6. Exit")

    choice = input("Enter your choice: ").lower()

    if choice == '1':
        countUsers(groupedData)

    elif choice == '2':
        countMovies(groupedData)

    elif choice == '3':
        user_id = int(input("Enter user ID: "))
        displayUser(groupedData, user_id)

    elif choice == '4':
        movie_id = int(input("Enter movie ID: "))
        displayMovie(movie_titles_dict, movie_id)

    elif choice == '5':
        user_id = int(input("Enter user ID for recommendations: "))
        num_recommendations = int(input("How many recommendations do you want?: "))
        recommendMovies(groupedData, user_id, num_recommendations, movie_titles_dict)

    elif choice == '6':
        print("Exiting...")
        break

    else:
        print("Invalid choice! Please select a valid option.")


Menu:
1. Count Users
2. Count Movies
3. Display User Information
4. Display Movie Name
5. Recommend Movies
6. Exit
Enter your choice: 1
Total users: 944

Menu:
1. Count Users
2. Count Movies
3. Display User Information
4. Display Movie Name
5. Recommend Movies
6. Exit
Enter your choice: 2
Total movies: 1682

Menu:
1. Count Users
2. Count Movies
3. Display User Information
4. Display Movie Name
5. Recommend Movies
6. Exit
Enter your choice: 3
Enter user ID: 284
Movies rated by User 284:
Movie: Fly Away Home (1996), Rating: 4
Movie: Murder at 1600 (1997), Rating: 3
Movie: Full Monty, The (1997), Rating: 4
Movie: Ice Storm, The (1997), Rating: 4
Movie: Oscar & Lucinda (1997), Rating: 3
Movie: Contact (1997), Rating: 4
Movie: George of the Jungle (1997), Rating: 2
Movie: Mad City (1997), Rating: 3
Movie: I Know What You Did Last Summer (1997), Rating: 3
Movie: Titanic (1997), Rating: 3
Movie: Apostle, The (1997), Rating: 4
Movie: Conspiracy Theory (1997), Rating: 4
Movie: Eve's Bayou (199